#### Пример свёрстки для распознавания цифр

In [20]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data


# Скачаем датасет MNIST:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

# Зададим заглушки для тренировочных данных:
x = tf.placeholder(tf.float32, [None, 784])
y = tf.placeholder(tf.float32, [None, 10])

# Переформатируем входной вектор в виде двумерного массива из 28 × 28 пикселов:
x_image = tf.reshape(x, [-1, 28, 28, 1]) # [размер батча, высота, ширина, каналы], −1 соответствует заранее неизвестному размеру мини-батча

# Создадим ядро свёрстки 5 × 5
W_conv_1 = tf.Variable(tf.truncated_normal([5, 5, 1, 32], stddev=0.1)) # [высота, ширина, входные каналы, выходные каналы]

# Задаём свободные члены (biases):
b_conv_1 = tf.Variable(tf.constant(0.1, shape=[32]))

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [22]:
# Теперь у нас определены переменные для всех весов сверточного слоя; что с ними делать дальше, TensorFlow знает сам:
conv_1 = tf.nn.conv2d(input=x_image, filter=W_conv_1, strides=[1, 1, 1, 1], padding='SAME') + b_conv_1 # strides просто определяет, как часто мы применяем фильтры по каждой размерности входного тензора

# В качестве функции активации возьмем ReLU:
h_conv_1 = tf.nn.relu(conv_1)

# Итак, слой фильтров с нелинейностью готов. Чтобы соблюсти стандартную архитектуру сверточной сети, осталось только добавить слой субдискретизации:
h_pool_1 = tf.nn.max_pool(value=h_conv_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# Функция tf.nn.max_pool определяет max-pooling слой, выбирая максимальное значение из каждого окна. Параметр ksize здесь как раз и задает размер этого окна,
# в котором мы выбираем максимальный элемент. Он имеет ту же структуру, что и strides. Обратите внимание, что здесь уже вполне можно представить себе ситуацию, 
# когда первая компонента будет не равна единице и мы захотим выбирать «самые подходящие» из нескольких последовательных изображений. 
# Можно даже задать первую размерность −1, тогда слой субдискретизации будет выбирать «самое подходящее» изображение из всего мини-батча.

# Параметры strides и padding обозначают здесь то же самое, что и для сверточного слоя, только в этот раз мы двигаемся по изображению в обе стороны с шагом 2.
# Понятно, что после этого слоя размер изображения в обоих направлениях уменьшится вдвое, до 14 × 14

In [25]:
# Раз весов у нас не так уж и много, давайте по той же схеме, что и выше, добавим еще один сверточный слой и слой субдискретизации, 
# в этот раз используя на этом слое 64 фильтра:

W_conv_2 = tf.Variable(tf.truncated_normal([5, 5, 32, 64], stddev=0.1))
b_conv_2 = tf.Variable(tf.constant(0.1, shape=[64]))

conv_2 = tf.nn.conv2d(input=h_pool_1, filter=W_conv_2, strides=[1, 1, 1, 1], padding='SAME') + b_conv_2
h_conv_2 = tf.nn.relu(conv_2)
h_pool_2 = tf.nn.max_pool(value=h_conv_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

In [38]:
# Как правило, в глубоких нейронных сетях за сверточными слоями следуют полносвязные, задача которых состоит в том, чтобы «собрать вместе» все признаки из фильтров и 
# собственно перевести их в самый последний слой, который выдаст ответ. Но для начала нам нужно из двумерного слоя сделать плоский; в TensorFlow это делается функцией reshape:

h_pool_2_flat = tf.reshape(h_pool_2, [-1, 7*7*64]) # Число 7*7*64 возникло из-за того, что мы дважды применили субдискретизацию и при этом в последнем слое использовали 64 фильтра. 

# Теперь осталось добавить полносвязные слои. Добавляем первый слой из 1024 нейронов:
W_fc_1 = tf.Variable(tf.truncated_normal([7*7*64, 1024], stddev=0.1))
b_fc_1 = tf.Variable(tf.constant(0.1, shape=[1024]))
h_fc_1 = tf.nn.relu(tf.matmul(h_pool_2_flat, W_fc_1) + b_fc_1)

# Регуляризуем его дропаутом:
keep_probability = tf.placeholder(tf.float32)
h_fc_1_drop = tf.nn.dropout(h_fc_1, keep_probability)

# Теперь добавляем второй, самый последний слой с десятью выходами:
W_fc_2 = tf.Variable(tf.truncated_normal([1024, 10], stddev=0.1))
b_fc_2 = tf.Variable(tf.constant(0.1, shape=[10]))

logit_conv = tf.matmul(h_fc_1_drop, W_fc_2) + b_fc_2
y_conv = tf.nn.softmax(logit_conv)

# Осталось определить ошибку и ввести оптимизатор:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logit_conv, labels=y))
train_step = tf.train.AdamOptimizer(0.0001).minimize(cross_entropy)

In [39]:
# Оцениваем точность:

correct_prediction = tf.equal(tf.argmax(y_conv, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [54]:
# И осталось только запустить обучение и дождаться результата

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(64)
    sess.run(train_step,
        feed_dict={x: batch_xs, y: batch_ys, keep_probability: 0.5})

In [55]:
print(sess.run(accuracy,
    feed_dict={x: mnist.test.images, y: mnist.test.labels, keep_probability: 1.}))

0.9893
